## ライブラリのインストール

In [0]:
# Catalyst
# ・githubの最新
!pip install git+git://github.com/catalyst-team/catalyst.git
# ・リリースバージョン
# !pip install -U catalyst

!pip install tensorflow
%load_ext tensorboard


## Google Driveのマウント

In [0]:
from google.colab import drive

gdrive_path = "/content/drive"
drive.mount(gdrive_path)

%cd "/content/drive/My Drive/cifar100"


## 学習データの準備

In [0]:
import collections
import torch
import torchvision
import torchvision.transforms as transforms

def get_loaders(datasets_path, batch_size, num_workers):
    normalize = transforms.Normalize(
        mean=[0.4914, 0.4824, 0.4467],
        std=[0.2471, 0.2435, 0.2616]
    )

    transform_train = transforms.Compose([
        transforms.Pad(4, padding_mode = 'reflect'),
        transforms.Resize(224),  # リサイズ
        transforms.RandomCrop(224),  # ランダムでピクセル欠け
        transforms.RandomHorizontalFlip(),  # ランダムで上下反転
        transforms.ToTensor(),  # テンソル化
        normalize
    ])
    transform_test = transforms.Compose([
        transforms.Resize(224),  # リサイズ
        transforms.ToTensor(),  # テンソル化
        normalize
    ])

    trainset = torchvision.datasets.CIFAR100(root=datasets_path, train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR100(root=datasets_path, train=False, download=True, transform=transform_test)

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory = True)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory = True)

    loaders = collections.OrderedDict()
    loaders["train"] = trainloader
    loaders["valid"] = testloader
    return loaders


## モデル定義

In [0]:
import torchvision.models as models

def get_model(model_name):
    return models.__dict__[model_name](num_classes=100)

print(models.__dict__.keys())


## SupervisedRunner生成

In [0]:
import os
import torch.nn as nn
import catalyst
import catalyst.contrib.optimizers as optim
from catalyst.dl.runner import SupervisedRunner
from catalyst.dl.callbacks import EarlyStoppingCallback, AccuracyCallback
from catalyst.dl import utils

# パラメータ
model_name = "resnet18"
num_epochs = 10
datasets_path = "./datasets"
batch_size = 32         # バッチサイズ
num_workers = 4         # 読込みのCPUコア数
lr = 0.001              # 学習率
logdir = "./logs/" + model_name
resume_file = logdir + "/checkpoints/best_full.pth"

# モデル生成
model = get_model(model_name)
criterion = nn.CrossEntropyLoss()

optimizer = optim.RAdam(model.parameters(), lr=lr)
loaders = get_loaders(datasets_path, batch_size, num_workers)

device = utils.get_device()
print("device: {}".format(device))

resume = None
if os.path.exists(resume_file):
    resume = resume_file
print("resume: {}".format(resume))

runner = SupervisedRunner(device=device)

## Tensorboard起動

In [0]:
%tensorboard --logdir {logdir}

## 学習開始

In [0]:
# 学習開始
runner.train(
    model=model,
    resume=resume,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    callbacks=[
        AccuracyCallback(accuracy_args=[1, 5])
    ],
    logdir=logdir,
    num_epochs=num_epochs,
    main_metric="accuracy01",
    minimize_metric=False,
    fp16=None,
    verbose=True
)